# Azted G-dDT Test Run

In [1]:
import sys
sys.path.append('../')

import pandas as pd
import os
import plotly.express as px
import seaborn as sns 
import matplotlib.pyplot as plt

import plotly.io as pio
pio.renderers.default = "png"

In [2]:
from aztec_gddt.experiment import standard_run

sim_df = standard_run()


ImportError: cannot import name 'standard_run' from 'aztec_gddt.experiment' (/Users/danlessa/repos/bsci/aztec-gddt-v2/notebooks/../aztec_gddt/experiment.py)

In [ ]:
sim_df.current_process.map(lambda x: x.leading_sequencer if x is not None else None).dropna().value_counts()

In [ ]:
agent_labels = [i for i in sim_df['agents'].iloc[0].keys() if type(i) != str]
x = sim_df.time_l1
fig_df = sim_df.copy()
for i in agent_labels:
    fig_df[f'agent_{i}_staked'] = sim_df.agents.map(lambda x: x[i].staked_amount)
    y_i = fig_df[f'agent_{i}_staked']
    plt.plot(x, y_i)

In [ ]:
sim_df.transactions.map(lambda x: len(x))

from aztec_gddt.types import *

fig_df['proposal_tx_count'] = sim_df.transactions.map(lambda tx_dict: len(list(v for v in tx_dict.values() if isinstance(v, Proposal))))
fig_df.proposal_tx_count.plot()

In [ ]:
fig_df.proposal_tx_count

In [ ]:
sns.lineplot(x=sim_df.timestep, y=sim_df.time_l1)

In [ ]:
sim_df.head(4)

In [ ]:
fig_df = sim_df.copy()
fig_df['process_order'] = fig_df.current_process.map(lambda x: x.phase.value if x is not None else None)
fig_df['process_label'] = fig_df.current_process.map(lambda x: x.phase.name if x is not None else None)
fig_df = fig_df.sort_values(['process_order', 'timestep'])#.query("time_l1 > 320 & time_l1 < 370")


plt.figure(figsize=(15, 5))
sns.lineplot(x=fig_df.time_l1, 
             y=fig_df.process_label)

plt.figure(figsize=(15, 5))
sns.scatterplot(x=fig_df.time_l1, 
             y=fig_df.process_label)
plt.show()

In [ ]:
fig_df.set_index('time_l1').sort_index().process_label

In [ ]:
sim_df.iloc[-1]

In [ ]:
sns.lineplot(x=sim_df.time_l1, 
             y=sim_df.finalized_blocks_count)

In [ ]:
sns.lineplot(x=sim_df.time_l1, 
             y=sim_df.cumm_block_rewards)

In [ ]:
sim_df['balance_sequencers'] = sim_df.agents.map(lambda x: sum(el.balance for el in x.values() if el.is_sequencer == True))
sim_df['balance_provers'] = sim_df.agents.map(lambda x: sum(el.balance for el in x.values() if el.is_prover == True))
sim_df['balance_relays'] = sim_df.agents.map(lambda x: sum(el.balance for el in x.values() if el.is_relay == True))
sim_df['balance_burnt'] = sim_df.agents.map(lambda x: sum(el.balance for el in x.values() if el.uuid == 'burnt'))
sim_df['balance_l1_builder'] = sim_df.agents.map(lambda x: sum(el.balance for el in x.values() if el.uuid == 'l1-builder'))


px.line(sim_df,
        x='time_l1',
        y=['balance_sequencers', 
           'balance_provers',
           'balance_relays',
           'balance_burnt',
           'balance_l1_builder'],
        log_y=True)

In [ ]:
px.scatter(sim_df,
        x='time_l1',
        y=['gas_fee_l1', 'gas_fee_blob'])

In [ ]:
px.histogram(sim_df,
        x=['gas_fee_l1'])

In [ ]:
px.histogram(sim_df,
        x=['gas_fee_blob'])

In [ ]:
sim_df.gas_fee_l1.min()

In [ ]:
ts_df = sim_df.token_supply.apply(lambda x: pd.Series(x.__dict__))
ts_df.plot(kind='line')
plt.title("Token Supply")
plt.show()

In [ ]:
ts_df = sim_df.token_supply.apply(lambda x: pd.Series(x.__dict__)).diff()
ts_df.plot(kind='line')
plt.title("Token Supply Difference")
plt.show()